In [1]:
import os
import re
import json
import pandas as pd
import numpy as np

In [2]:
with open('./data/class_doe.json', 'r') as f:
    name_to_code = json.load(f)

code_to_name = {
    value : key for key, value in name_to_code.items()
}

In [3]:
#[잡담] < 말머리 삭제
def replace_txt(s):
    return re.sub('(?=\[).+(?<=\])','', s)

def str_to_list(s):
    return s.split(',')

def comment_to_strucks(s):
    result = []
    for sample in s[1:-2].split("',"):
        t = re.sub('\\\\n+',' ',sample.strip()[1:])
        result.append(t)
    
    return result

In [4]:
base_path = './data/'
file_names = os.listdir(base_path)

df_lst = []
# concat_articles
for file in file_names:
    if not 'csv' in file:
        continue
    tmp = pd.read_csv(base_path+file)
    tmp['comments'] = tmp['comments'].apply(comment_to_strucks)
    tmp['title'] = tmp['title'].apply(replace_txt)
    df_lst.append(tmp)

df = pd.concat(df_lst)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5416 entries, 0 to 5415
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   num                5416 non-null   int64 
 1   title              5416 non-null   object
 2   content            5406 non-null   object
 3   writed_at          5409 non-null   object
 4   comments           5416 non-null   object
 5   comment_writed_at  5416 non-null   object
 6   url                5416 non-null   object
 7   classcode          5416 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 338.6+ KB


>Warrior
- 디트 : 00
- 워로드 : 01
- 버서커 : 02
- 홀리나이트 : 03
- 슬레이어 : 04

>Martial_Artist
- 인파 : 00
- 배마 : 01
- 기공 : 02
- 창술 : 03
- 스커 : 04

>Gunner
- 스카우터 : 00
- 호크 : 01
- 블래스터 : 02
- 데빌헌터 : 03
- 건슬 : 04

>Mage
- 서머너 : 00
- 아르카나 : 01
- 소서리스 : 02
- 바드 : 03

>Assassin
- 데모닉 : 00
- 블레이드 : 01
- 리퍼 : 03

>Specialist.
- 기상 : 00
- 도화가 : 01

In [6]:
test_df = df[df['classcode'] == name_to_code['창술사']]

In [14]:
with open('./data/txt/Martial_Artist_03.txt', 'w', encoding='utf8') as f:
    for title, content, comments in zip(test_df['title'].values, test_df['content'].values, test_df['comments'].values):

        f.write(title + '\n')

        try:
            if content != '':
                f.write(content + '\n')
        except TypeError as TE:
            print(content,  TE)
            pass

        for comment in comments:
            try:
                if (comment != ''):
                    f.write(comment + '\n')
            except TypeError as TE:
                print(comment, TE)
                pass




nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'
nan unsupported operand type(s) for +: 'float' and 'str'


In [15]:
with open('./data/txt/Martial_Artist_03.txt', 'r', encoding='utf8') as f:
    test_txt = f.read().split('\n')


In [16]:
len(test_txt)

11901

In [20]:
input_file = './data/txt/Martial_Artist_03.txt'
vocab_size= 5000
prefix = './data/txt/sentencepiece/nsmc'

In [21]:
templates = '--input={} --model_prefix={} --vocab_size={}'
cmd = templates.format(input_file, prefix, vocab_size)
cmd

'--input=./data/txt/Martial_Artist_03.txt --model_prefix=./data/txt/sentencepiece/nsmc --vocab_size=5000'

In [22]:
import sentencepiece as spm

In [23]:
spm.SentencePieceTrainer.Train(cmd)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=./data/txt/Martial_Artist_03.txt --model_prefix=./data/txt/sentencepiece/nsmc --vocab_size=5000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./data/txt/Martial_Artist_03.txt
  input_format: 
  model_prefix: ./data/txt/sentencepiece/nsmc
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id

In [24]:
sp = spm.SentencePieceProcessor()

In [25]:
sp.Load('{}.model'.format(prefix))

True

In [27]:
for t in test_df['title'].values[:10]:
    print(t)
    print(sp.encode_as_pieces(t))
    print()

절제 개인적인 세팅 공유
['▁절제', '▁개인적', '인', '▁세팅', '▁공유']

밸패전 세팅 특신이어도 치확100%트포 빼는게 맞을까요?
['▁밸패전', '▁세팅', '▁특신', '이', '어도', '▁치확', '100%', '트포', '▁빼', '는게', '▁맞을까요', '?']

애니츠는 이게 맞아
['▁애니츠', '는', '▁이게', '▁맞아']

1613 절정창술 특치인가요 특신인가요?
['▁16', '1', '3', '▁절정창술', '▁특치', '인가요', '▁특신', '인가요', '?']

각인 특성치좀 봐주실분?
['▁각인', '▁특성', '치', '좀', '▁봐', '주실분', '?']

극특치 절정 팔찌 질문좀요.
['▁극특치', '▁절정', '▁팔찌', '▁질문좀', '요', '.']

돌대저받97돌 정배가 뭐에요?
['▁돌대', '저받', '97', '돌', '▁정배', '가', '▁뭐', '에', '요', '?']

창술 두껍쉑 딜 미쳤네요…우매봉 머쓱…
['▁창술', '▁두', '껍', '쉑', '▁딜', '▁미쳤', '네요', '...', '우', '매', '봉', '▁머', '쓱', '...']

유물+고대섞어서 333331 하는건어떤가요?
['▁유물', '+', '고', '대', '섞', '어서', '▁333331', '▁', '하는건', '어', '떤', '가요', '?']

절제는 이제 일섬각 안넣나요
['▁절제는', '▁이제', '▁일섬각', '▁안', '넣', '나요']

